# Extract Data from [Football API](https://www.api-football.com/documentation-v3#tag/Players/operation/get-players)

*See player_stats.py for details on functions used below*

## Create stats dataset for the following leagues

In [ ]:
import player_stats
# Leagues: Premier League, Serie A, La Liga, Bundesliga, Ligue 1, MLS, Eredivisie
# Championship, Primeria Liga, Super Lig
leagues = ["39"] # "135", "140", "78", "61", "866", "88", "40", "94", "203"

### 1. Pull and save all necessary data from API

The below code will save json files to the current directory called "seasons.json", "teams.json" and "players.json".

Only call this funciton get/update the data in the saved files. This allows us to have access to the data locally without making multiple API calls each time we need the data.

In [ ]:
###########################################
#  RUN THIS ONCE TO UPDATE THE JSON FILES #
###########################################

player_stats.update_files(leagues)

###########################################
#  RUN THIS ONCE TO UPDATE THE JSON FILES #
###########################################

### 2. Load data

In [ ]:
import json

# load seasons
with open("seasons.json", "r") as file:
    seasons = file.readlines()
    seasons = [year.strip("\n") for year in seasons]
    
# load teams
# with open("teams.json", "r") as file:
#     teams = json.loads(file.read())

# load players
with open("players.json", "r") as file:
    players = json.loads(file.read())

print("Num leagues:", len(leagues))
print("Num seasons:", len(seasons))
# print("Num teams per league and season:", len(teams))
print("Num players per league season:", len(players))